# Simulate Dynamics for example with SQS

1) Simulate neutral and species-model dynamics
2) Calculate SQS metrics in R with SQSAnalysis.R
3) Import results into notebook and conduct Haar fluctuation analysis and plot

In [ ]:
# == Library Import == 
# Set directrory to src
# Run a simulation and save the output matrix.
src_paths = "../src/"

# Set paths 
import os
import sys
os.chdir(src_paths)
sys.path.append(os.getcwd()) 

# From src
## Diversity Metric Functions
from diversityMetrics import *
import divDynFunctions
from pareto import *
import untbPython
import samplingFunctions
import divDynFunctions
## Haar Fluctuation Analysis Functions
from haarFluctuationAnalysis import Haar_hebert
from crossHaarCorrelation import CHFC

# Other Imports
import pandas as pd
import numpy as np
from itertools import product
from numba import njit, set_num_threads

# Depositional Model
def depositional_model_time(cutoffs, max_time, deposition_integration):

    # Externally Set Parameters
    cutoff = cutoffs # This is what we will be setting for the pareto_cutoff

    # Internally Set Parameters
    gamma = 0.4 # Shape parameter used in Schumer and Jerolmack 2009
    xmin = 1 # minimum hiatus duration (year); using integer values so must be int
    deposition_rate = 1000  # cm/yr
    deposition_duration = deposition_integration  # 1000 years
    deposition_thickness = deposition_rate * deposition_duration  # deposition = 10 cm
    max_time = max_time  # max total simulated time in kyr (stop condition); Base on selection from UNTB simulation

    # Set Save Directories
    times = [0]
    elevation = [0]
    obs_record =[]
    t = 0
    s = 0
    step = 0

    while t < max_time:
        # Setting the hiatuses
        while True:
            hiatus = rpareto(n=1, lam=gamma, a=xmin).astype(int)[0]
            if hiatus <= cutoff:
                break

        # Check if next step would exceed max_time
        if t + hiatus + deposition_duration > max_time:
            break 

        # Record dynamics within step
        t += hiatus
        obs_record.extend([False] * hiatus)  # Append erosion / nondeposition
        t += deposition_duration
        obs_record.extend([True] * deposition_duration)  # Append deposition
        s += deposition_thickness
        times.append(t)
        elevation.append(s)
        step += 1

    # Will return up to the while statement
    return np.asarray(obs_record), np.asarray(times), np.asarray(elevation)

# Neutral Model
# Redefintion of UNTB function to include actual number of mutations per individual time step
@njit
def untb_Hankin_Phylo_V2(initial_population, mutation_probability, death_rate, generations, keep = False):
    # Initalize
    pop_size = len(initial_population) # Size of Jm
    
    # Allocate output matrix
    out_matrix = np.full(shape = (generations, pop_size), fill_value = np.nan)    
    a = initial_population.copy()
    out_matrix[0,:] = a # Time 1 is initial population

    # Speciation Tracker
    ansector = np.full(shape = (generations, death_rate), fill_value = np.nan)    
    descendent = np.full(shape = (generations, death_rate), fill_value = np.nan)   

    # Tracker number of new mutations
    total_mutations = [0]

    # Max Species Tracker
    max_species_id = np.max(a)

    # Simulate
    for i in range(1, generations):
    
        died = np.random.choice(pop_size, death_rate, replace = False) # index position of deaths
        mutated = np.random.uniform(0.0, 1, size=len(died)) < mutation_probability # probability new mutation arises
        n_mutations = np.sum(mutated) # new species
        n_deaths = np.sum(~mutated) # replaced species
        total_mutations.append(n_mutations)

        if n_deaths > 0:
            a[died[~mutated]] = np.random.choice(a, n_deaths, replace = True)
        if n_mutations > 0:
            new_species = np.arange(1, n_mutations + 1) + max_species_id 
            max_species_id += n_mutations

            ansector[i,0:n_mutations] = a[died[mutated]]
            a[died[mutated]] = new_species 
            descendent[i,0:n_mutations] = new_species
            
        out_matrix[i,:] = a
        
    return out_matrix, ansector, descendent, np.asarray(total_mutations)

In [ ]:
## == Define Parameters ==
num_runs = 1

# Define Parameter Grid
pareto_cutoff = [10, 100, 1000]
deposition_integration = [1, 10, 100]
param_grid = list(product(pareto_cutoff, deposition_integration))

## == Begin Simulations ==
sim_length = 12000 # Running the simulation for 50,000 time steps
J = 100 # Community size 
v = 0.05 # Speciation rate
d = 10
cutoffs = np.array([1000])
model_steps = np.max(cutoffs) + sim_length # Extending the simulation time
time_v = np.arange(cutoffs[0], cutoffs[0] + sim_length) - cutoffs[0]

# Simulate Neutral Ecology
expanded_community = np.repeat(1,J)
untb_results, ancs, desc, _ = untb_Hankin_Phylo_V2(expanded_community, mutation_probability = v, death_rate = d, generations = model_steps, keep = False)

# Make sure we have a simulation that doesn't go extinct
species_list_all = np.arange(1, np.max(untb_results)+1, dtype = np.int64)
set_num_threads(8) # Set thread number
spec_dyn = samplingFunctions.retrieve_species_dynamics_numba(untb_results, species_id = species_list_all)

## == Metrics for the perfectly sampled community using second-for-third == 
# Ideal Haar fluctuations for the community 
cut = cutoffs[0]
# Bias the output
test_spec = np.array(spec_dyn.copy(), dtype=np.int64)
presence = test_spec[cut:] # Discard burn-in period
# Get metrics
# Filter out time steps you will not observe
occ_list = convert_to_dictionary(presence)

# Diversity 
dynMat = divDynFunctions.binnedToExpandedFad(occ_list)
metricMatrix = divDynFunctions.counts(dynMat)

In [ ]:
# Save metricMatrix
# Example if it's an array
df = pd.DataFrame(dynMat)
df.to_csv("./data/ForDivDyn/dynMat.csv", index=False)

In [ ]:
## == Define Parameters ==
num_runs = 1

# Define Parameter Grid
pareto_cutoff = [1000]
deposition_integration = [10]
param_grid = list(product(pareto_cutoff, deposition_integration))

## == Begin Simulations ==
sim_length = 12000 # Running the simulation for 50,000 time steps
J = 100 # Community size 
v = 0.05 # Speciation rate
d = 10
cutoffs = np.array([1000])
model_steps = np.max(cutoffs) + sim_length # Extending the simulation time
time_v = np.arange(cutoffs[0], cutoffs[0] + sim_length) - cutoffs[0]

# Simulate Neutral Ecology
expanded_community = np.repeat(1,J)
untb_results, ancs, desc, _ = untb_Hankin_Phylo_V2(expanded_community, mutation_probability = v, death_rate = d, generations = model_steps, keep = False)

# Make sure we have a simulation that doesn't go extinct
species_list_all = np.arange(1, np.max(untb_results)+1, dtype = np.int64)
set_num_threads(8) # Set thread number
spec_dyn = samplingFunctions.retrieve_species_dynamics_numba(untb_results, species_id = species_list_all)


## == Run Model == ##

# Run intial simulation for data before aggregation
# Set desktop path
results_summary = []

for cutoff, depositional_steps in param_grid:
    save_comb = []

    for run_id in range(1, num_runs + 1):
        try:
            print(f"Running simulation with cutoff={cutoff}, dep={depositional_steps}")

            ###
            # Retrive UNTB outputs
            test_spec = np.array(spec_dyn.copy(), dtype=np.int64)
            presence = test_spec[cutoffs[0]:]
            occ_list = convert_to_dictionary(presence[time_v,:])
            o_div = [len(i) for i in occ_list]

            #### -------------------------------------
            # Run depositional model
            obs_record, times, elevation = depositional_model_time(cutoffs = cutoff, max_time = 12000, deposition_integration = depositional_steps)

            # Bias the output
            # Set all arrays to empty if erosion occurs
            for i in range(len(obs_record)):
                state = obs_record[i]
                if state == False:
                    occ_list[i] = np.array([])

            # Filter based on hiatuses and plot
            o_div_filt = [len(i) for i in occ_list]
            non_zero_values = [val for val in o_div_filt if val > 0]
            #min_non_zero = min(non_zero_values) if non_zero_values else 0

            # Combine elements of array based on boundaries
            new_occ_list = []
            for i in range(len(times) - 1):
                agg_list = occ_list[int(times[i]):int(times[i+1])]
                if len(agg_list) <= 1:
                    new_occ_list.append(np.array(list(agg_list)))
                else:
                    aggregated_beds = np.unique(np.concatenate(agg_list)) 
                    new_occ_list.append(aggregated_beds)

            # Diversity Metrics
            # Diversity 
            dynMat = divDynFunctions.binnedToExpandedFad(new_occ_list)

            # Map Times 
            mid_point_times = (times[:-1] + np.diff(times)/2)

            mapped_values = mid_point_times[dynMat[:, 1]]
            arr_new = np.column_stack((dynMat, mapped_values))

            # Save metricMatrix
            # Example if it's an array
            save_name = f"sim_{cutoff}_{depositional_steps}"
            df = pd.DataFrame(arr_new)
            df.to_csv(f"./data/ForDivDyn/{save_name}.csv", index=False)

        except Exception as e:
            print(f"Error for Pareto cutoff={cutoff}, depositional steps = {depositional_steps}, run={run_id}: {e}")

print(f"Simulations complete.")


Running simulation with cutoff=1000, dep=10
Simulations complete.
